In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

In [10]:
# Specify the file name
file_name = 'df_OptionB.csv'

# Specify the full file path
file_path = r'C:\Users\jeroe\OneDrive\Documenten\3. VU\Master Econometrics\Data Mining Techniques\Assignment 2\\' + file_name

# Read the dataset using pandas
df = pd.read_csv(file_path)

In [11]:
df.columns

Index(['srch_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'click_bool', 'booking_bool', 'score',
       'avg_comp_rate', 'avg_comp_inv', 'abroad_bool', 'children_bool',
       'srch_query_affinity_score_low', 'srch_query_affinity_score_high',
       'starrating_diff_low', 'starrating_diff_high', 'usd_diff_low',
       'usd_diff_high'],
      dtype='object')

In [12]:
click_indices = df[df['click_bool'] == 1].index
random_indices = np.random.choice(click_indices, len(df.loc[df['click_bool'] == 1]), replace=False)
click_sample = df.loc[random_indices]

not_click = df[df['click_bool'] == 0].index
random_indices = np.random.choice(not_click, sum(df['click_bool']), replace=False)
not_click_sample = df.loc[random_indices]

df = pd.concat([not_click_sample, click_sample], axis=0)

print("Non-click impressions: ", len(df[df['click_bool'] == 0]))
print("Click impression: ", len(df[df['click_bool'] == 1]))
print("Total number of records in resampled data: ", len(df))

Non-click impressions:  221879
Click impression:  221879
Total number of records in resampled data:  443758


In [13]:
df = df.sort_values(by='srch_id')

y = df['score']
X = df.drop(['score'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

group_train = list(X_train['srch_id'].value_counts())
group_test = list(X_test['srch_id'].value_counts())

X_train = X_train.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)
X_test = X_test.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)

In [16]:

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        # Add other hyperparameters to be tuned
    }

    gbm = lgb.LGBMRanker(**params)
    gbm.fit(X_train, y_train, group=group_train,
            eval_set=[(X_train, y_train), (X_test, y_test)], 
            eval_group=[group_train, group_test],
            eval_at=[5], early_stopping_rounds=50, verbose=False)

    # Evaluate on the test set
    test_preds = gbm.predict(X_test)
    test_ndcg = ndcg_score([y_test], [test_preds], k=5)

    return test_ndcg

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters and retrain the model
best_params = study.best_params
gbm_best = lgb.LGBMRanker(**best_params)
gbm_best.fit(X_train, y_train, group=group_train,
             eval_set=[(X_train, y_train), (X_test, y_test)],
             eval_group=[group_train, group_test],
             eval_at=[5], early_stopping_rounds=50, verbose=False)

[I 2023-05-10 13:56:09,520] A new study created in memory with name: no-name-dbcde1fc-75c8-4d6e-83fa-afdca2cf94f0
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. S

[LightGBM] [Warning] feature_fraction is set=0.8438353479485673, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8438353479485673
[LightGBM] [Warning] bagging_fraction is set=0.591876823914401, subsample=1.0 will be ignored. Current value: bagging_fraction=0.591876823914401


[I 2023-05-10 13:56:14,128] Trial 1 finished with value: 0.9999999999999999 and parameters: {'num_leaves': 70, 'learning_rate': 0.012353120962924504, 'feature_fraction': 0.8438353479485673, 'bagging_fraction': 0.591876823914401}. Best is trial 1 with value: 0.9999999999999999.
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
C:\User

[LightGBM] [Warning] feature_fraction is set=0.7344021526839386, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344021526839386
[LightGBM] [Warning] bagging_fraction is set=0.572967555696965, subsample=1.0 will be ignored. Current value: bagging_fraction=0.572967555696965


[I 2023-05-10 13:56:16,806] Trial 2 finished with value: 0.9999999999999999 and parameters: {'num_leaves': 148, 'learning_rate': 0.046817458689835716, 'feature_fraction': 0.7344021526839386, 'bagging_fraction': 0.572967555696965}. Best is trial 1 with value: 0.9999999999999999.
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
C:\Use

...

[I 2023-05-10 14:01:29,075] Trial 98 finished with value: 0.9999999999999999 and parameters: {'num_leaves': 97, 'learning_rate': 0.07035793257801827, 'feature_fraction': 0.7293709751973304, 'bagging_fraction': 0.5296593550913469}. Best is trial 80 with value: 1.0.
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\jeroe\AppData\Local\Temp\ipykernel_24508\4010859408.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
C:\Users\jeroe\AppDa

[LightGBM] [Warning] feature_fraction is set=0.7974894843445701, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7974894843445701
[LightGBM] [Warning] bagging_fraction is set=0.6553414158905553, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6553414158905553


[I 2023-05-10 14:01:32,417] Trial 99 finished with value: 0.8831453201258348 and parameters: {'num_leaves': 86, 'learning_rate': 0.013745996405742983, 'feature_fraction': 0.7974894843445701, 'bagging_fraction': 0.6553414158905553}. Best is trial 80 with value: 1.0.
c:\Users\jeroe\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\jeroe\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.8133172415670514, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8133172415670514
[LightGBM] [Warning] bagging_fraction is set=0.6139375318984629, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6139375318984629


LGBMRanker(bagging_fraction=0.6139375318984629,
           feature_fraction=0.8133172415670514,
           learning_rate=0.011208792951384421, num_leaves=78)

In [30]:
### apply model to real test set
file_name = 'df_testB.csv'
file_path = r'C:\Users\jeroe\OneDrive\Documenten\3. VU\Master Econometrics\Data Mining Techniques\Assignment 2\\' + file_name
df_test = pd.read_csv(file_path)

In [31]:
df_test.columns

Index(['srch_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'avg_comp_rate', 'avg_comp_inv',
       'abroad_bool', 'children_bool', 'srch_query_affinity_score_low',
       'srch_query_affinity_score_high', 'starrating_diff_low',
       'starrating_diff_high', 'usd_diff_low', 'usd_diff_high'],
      dtype='object')

In [32]:
# Sort test set 
df_test = df_test.sort_values(by='srch_id')

# Creating group list to use in test set prediction
group = list(df_test['srch_id'].value_counts())

# Storing the removed columns in a separate DataFrame
df_test_columns_removed = df_test[['srch_id', 'prop_id']].copy()

# Create a new DataFrame by dropping the desired columns
df_test = df_test.drop(columns=['srch_id', 'prop_id'])

In [33]:
df_test.columns

Index(['prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'avg_comp_rate', 'avg_comp_inv',
       'abroad_bool', 'children_bool', 'srch_query_affinity_score_low',
       'srch_query_affinity_score_high', 'starrating_diff_low',
       'starrating_diff_high', 'usd_diff_low', 'usd_diff_high'],
      dtype='object')

In [34]:
# Apply model to test set
pred = gbm_best.predict(df_test, group=group)

In [35]:
# Add predictions to test set
df_test["predicted_ranking"] = pred

# Adding the columns back
df_test['srch_id'] = df_test_columns_removed['srch_id']
df_test['prop_id'] = df_test_columns_removed['prop_id']

# Sort the dataframe by `srch_id` and `predicted_ranking` in descending order
test_sorted = df_test.sort_values(['srch_id', 'predicted_ranking'], ascending=[True, False])

In [36]:
# Select the columns `srch_id` and `prop_id`
result = test_sorted[['srch_id', 'prop_id']]
result

,srch_id,prop_id
9,1,54937
23,1,99484
12,1,61934
5,1,28181
17,1,78599
...,...,...
4959178,332787,32019
4959177,332787,29018
4959182,332787,99509
4959181,332787,94437


In [37]:
result.to_csv('LambdaMart_result.csv', index=False)